# 📘 10_load_videos_static_to_bigquery

### 🎯 Objetivo

Este notebook no transforma datos.  
Su única responsabilidad es: Cargar df_videos_static (generado en el notebook 04) hacia BigQuery como tabla histórica.  
Destino: youtube-datasets-360.angelgarciadatablog.videos_static

In [2]:
from dotenv import load_dotenv
import os
from google.cloud import bigquery

In [3]:
PROJECT_ID = os.getenv("GCP_PROJECT")
DATASET_ID = "angelgarciadatablog"
TABLE_ID = "videos_static"

FULL_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.{TABLE_ID}"

client = bigquery.Client(project=PROJECT_ID)

print("Destino configurado:", FULL_TABLE_ID)

Destino configurado: youtube-datasets-360.angelgarciadatablog.videos_static


## 🧱 Cargar snapshot desde Parquet (temporal)

⚠️ Nota temporal:
Durante la fase de notebooks, el DataFrame se carga desde Parquet como mecanismo de intercambio entre notebooks.
En la versión productiva (scripts .py), el DataFrame se pasará directamente sin almacenamiento intermedio.

In [7]:
from pathlib import Path
import pandas as pd

PROJECT_ROOT = Path.cwd().parents[0]
PROCESSED_PATH = PROJECT_ROOT / "data" / "processed" / "youtube"

file_path = PROCESSED_PATH / "videos_static.parquet"

print("Ruta:", file_path)
print("Existe:", file_path.exists())

df_videos_static = pd.read_parquet(file_path)

df_videos_static.head()


Ruta: /Users/angelgarciachanga/repositorios/publico/youtube-v3-data-pipeline/data/processed/youtube/videos_static.parquet
Existe: True


,video_id,channel_id,title,description,category_id,published_at,duration_seconds,thumbnail_url,video_url,extracted_at
0,xB4ecIksJSY,UCUEOHBht8pnQhQvCfIcl-gg,4. Git Push - Actualizar cambios de un reposit...,,27,2026-01-24 12:04:21+00:00,960,https://i.ytimg.com/vi/xB4ecIksJSY/maxresdefau...,https://www.youtube.com/watch?v=xB4ecIksJSY,2026-02-14 22:02:51.077435+00:00
1,7bwkNrRpgw0,UCUEOHBht8pnQhQvCfIcl-gg,Modo oscuro - Big query - Google Cloud,,27,2026-01-23 06:52:23+00:00,69,https://i.ytimg.com/vi/7bwkNrRpgw0/maxresdefau...,https://www.youtube.com/watch?v=7bwkNrRpgw0,2026-02-14 22:02:51.077435+00:00
2,HDyKUodeuNw,UCUEOHBht8pnQhQvCfIcl-gg,Como seleccionar valores iguales en SQL,,27,2026-01-23 06:43:39+00:00,294,https://i.ytimg.com/vi/HDyKUodeuNw/maxresdefau...,https://www.youtube.com/watch?v=HDyKUodeuNw,2026-02-14 22:02:51.077435+00:00
3,Zj6uiqMvFOU,UCUEOHBht8pnQhQvCfIcl-gg,3. Cómo funciona el bucle For en python - Repa...,,27,2026-01-17 20:07:55+00:00,1186,https://i.ytimg.com/vi/Zj6uiqMvFOU/maxresdefau...,https://www.youtube.com/watch?v=Zj6uiqMvFOU,2026-02-14 22:02:51.077435+00:00
4,RiYjYfMTGvw,UCUEOHBht8pnQhQvCfIcl-gg,2. Qué es el PATH en la instalación de python,,27,2026-01-11 23:25:57+00:00,1138,https://i.ytimg.com/vi/RiYjYfMTGvw/maxresdefau...,https://www.youtube.com/watch?v=RiYjYfMTGvw,2026-02-14 22:02:51.077435+00:00


In [8]:
df_videos_static.dtypes

video_id                            str
channel_id                          str
title                               str
description                         str
category_id                         str
published_at        datetime64[us, UTC]
duration_seconds                  int64
thumbnail_url                       str
video_url                           str
extracted_at        datetime64[us, UTC]
dtype: object

## 🏗 Crear tabla particionada con el esquema y datos del dataframe 




In [9]:
from google.api_core.exceptions import NotFound
from google.cloud.bigquery import SchemaField

schema = [
    SchemaField("video_id", "STRING"),
    SchemaField("channel_id", "STRING"),
    SchemaField("title", "STRING"),
    SchemaField("description", "STRING"),
    SchemaField("category_id", "STRING"),
    SchemaField("published_at", "TIMESTAMP"),
    SchemaField("duration_seconds", "INT64"),
    SchemaField("thumbnail_url", "STRING"),
    SchemaField("video_url", "STRING"),
    SchemaField("extracted_at", "TIMESTAMP"),
]

try:
    client.get_table(FULL_TABLE_ID)
    print("Tabla ya existe.")
    
except NotFound:
    table = bigquery.Table(FULL_TABLE_ID, schema=schema)
    client.create_table(table)
    print("Tabla creada.")


Tabla creada.


## 📌 Cargar datos del parquet a big query

In [10]:
# 2️⃣ Carga los datos desde tu DataFrame hacia BigQuery. WRITE TRUNCATE = sobreescribe los datos
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE"
)

job = client.load_table_from_dataframe(
    df_videos_static,
    FULL_TABLE_ID,
    job_config=job_config
)

job.result()

print("Tabla videos_static reemplazada correctamente.")


/Users/angelgarciachanga/repositorios/publico/youtube-v3-data-pipeline/venv/lib/python3.11/site-packages/google/cloud/bigquery/_pandas_helpers.py:484: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


Tabla videos_static reemplazada correctamente.
